In [12]:
# PySMT - for SAT solving
import pysmt.shortcuts as pysmtsol # Symbol, And, Not, Xor, is_sat


In [13]:
# PyEDA - for circuit benchmark creation and cnf generation 
import pyeda
import pyeda.boolalg.expr as eda # exprvar, Or, And, Xor, Not, expr2dimacscnf

In [74]:
from pysmt.smtlib.parser import open_
from pysmt.shortcuts import FreshSymbol, Symbol


def read_dimacs(fname):
    """Read a DIMACS CNF file from the given file.

    Returns a tuple: (vars_cnt, clauses, comments)
    """
    prob_type, vars_cnt, clauses_cnt = None, None, None
    max_var = 0
    comments = []
    clauses = []

    with open_(fname) as fin:
        for line in fin:
            if line[0] == "c":
                comments.append(line)
            elif line[0] == "p":
                _, prob_type, vars_cnt, clauses_cnt = line.split(" ")
                prob_type = prob_type.strip()
                if prob_type != "cnf":
                    raise IOError("File does not contain a cnf.")
                vars_cnt = int(vars_cnt.strip())
                clauses_cnt = int(clauses_cnt)
                break

        for line in fin:
            if line[0] == "c":
                comments.append(line)
            else:
                # TODO: More robust parsing of clauses
                cl = line.strip().split(" ")
                assert cl[-1].strip() == "0", cl
                clause = [int(lit) for lit in cl[:-1]]
                max_var = max(max_var, max(abs(lit) for lit in clause))
                assert not any(lit == 0 for lit in clause), clause
                clauses.append(clause)

    # Validation
    if clauses_cnt != len(clauses):
        raise IOError("Mismatch between declared clauses (%d) " % clauses_cnt +
                      "and actual clauses (%d) in DIMACS file." % len(clauses))
    if max_var > vars_cnt:
        raise IOError("Mismatch between declared variables (%d) " % vars_cnt +
                      "and actual variables (%d) in DIMACS file." % max_var)

    return vars_cnt, clauses, comments


def dimacs_to_pysmt(vars_cnt, clauses, comments):
    """Convert a DIMACS structure into a pySMT formula.

    Returns (formula, symbol_table). The symbol_table contains a
    mapping from pySMT symbol to DIMACS var_idx.

    """
    st = {}
    rev_st = {}
    for i in range(1, vars_cnt+1):
        # s = FreshSymbol(template=("_dimacs_%d"%i))
        s = Symbol(name=("_dimacs_%d"%i))
        st[i] = s
        st[-i] = pysmtsol.Not(s)
        rev_st[s] = i
    res = pysmtsol.And(pysmtsol.Or(st[lit] for lit in clause) \
              for clause in clauses)
    return res, rev_st

In [75]:
def read_dimacs_bench(circuit, benchname):
    vars_cnt, clauses, comments = read_dimacs(f'cnf_bench/{circuit}/{benchname}.cnf')
    cir, rev_st = dimacs_to_pysmt(vars_cnt, clauses, comments)
    print("read dimacs :", cir)
    print("symbol table: ", rev_st)
    return cir

In [76]:
def create_miter(impl_cir, spec_cir):
    miter = pysmtsol.Xor(impl_cir, spec_cir)
    return miter

In [77]:
def check_sat(miter):
    sat_res = pysmtsol.is_sat(miter)
    return sat_res

In [78]:
def create_intermediate_miters(circuit, out1, out2):
    impl_cir = read_dimacs_bench(circuit, out1)
    spec_cir = read_dimacs_bench(circuit, out2)
    miter_cir = create_miter(impl_cir, spec_cir)
    return miter_cir

    
def equivalence_check(circuit, bench1, bench2, out):
    print("bench1 : ", bench1, "  bench2 :", bench2)
    miter_lst = [create_intermediate_miters(circuit, bench1[i], bench2[i]) for i in range(out)]
    print("miter_out: ", miter_lst)
    miter_or = pysmtsol.Or(miter_lst)    

    print("miter : ", miter_or)

    sat_res = check_sat(miter_or)
    print("sat_res: ", sat_res)
    if sat_res: print(f"{bench1} and {bench2} are SAT --> UNEQUAL CIRCUITS")
    else: print(f"{bench1} and {bench2} are UNSAT --> EQUIVALENT CIRCUITS")
    return sat_res

In [80]:
circuit = "fullAdder"
circuit1 = ["sum1"]
circuit2 = ["cout2"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)
 


bench1 :  ['sum1']   bench2 : ['cout2']
read dimacs : ((_dimacs_1 | _dimacs_2 | _dimacs_3) & (_dimacs_2 | (! _dimacs_3) | (! _dimacs_1)) & (_dimacs_1 | (! _dimacs_3) | (! _dimacs_2)) & (_dimacs_3 | (! _dimacs_1) | (! _dimacs_2)))
symbol table:  {_dimacs_1: 1, _dimacs_2: 2, _dimacs_3: 3}
read dimacs : ((_dimacs_1 | _dimacs_3) & (_dimacs_2 | _dimacs_3) & (_dimacs_1 | _dimacs_2))
symbol table:  {_dimacs_1: 1, _dimacs_2: 2, _dimacs_3: 3}
miter_out:  [(! (((_dimacs_1 | _dimacs_2 | _dimacs_3) & (_dimacs_2 | (! ...) | (! ...)) & (_dimacs_1 | (! ...) | (! ...)) & (_dimacs_3 | (! ...) | (! ...))) <-> ((_dimacs_1 | _dimacs_3) & (_dimacs_2 | _dimacs_3) & (_dimacs_1 | _dimacs_2))))]
miter :  (! (((_dimacs_1 | _dimacs_2 | _dimacs_3) & (_dimacs_2 | (! ...) | (! ...)) & (_dimacs_1 | (! ...) | (! ...)) & (_dimacs_3 | (! ...) | (! ...))) <-> ((_dimacs_1 | _dimacs_3) & (_dimacs_2 | _dimacs_3) & (_dimacs_1 | _dimacs_2))))
sat_res:  True
['sum1'] and ['cout2'] are SAT --> UNEQUAL CIRCUITS


In [81]:
def circuit2dimacs(bool_formula):
    ex = bool_formula.to_cnf()
    exp, dimacs_bench = eda.expr2dimacscnf(ex)
    print("dimacs : ", dimacs_bench)
    return dimacs_bench

def write_dimacs(dimacs_bench, circuit, benchname):
    f = open(f"cnf_bench/{circuit}/{benchname}.cnf", "w")
    f.write(str(dimacs_bench))
    f.close()

def write_circuit(bool_formulae, circuit, benchnames):
    for i,formula in enumerate(bool_formulae):
        dimacs_ = circuit2dimacs(formula)
        write_dimacs(dimacs_, circuit, benchnames[i])
        


In [83]:
## Adder
circuit = "fullAdder"
a, b, ci = map(eda.exprvar, "a b ci".split())

# Circuit 1: 
sum1 =  eda.Xor('a', 'b', 'ci')
cout1 = eda.Or(eda.And('a', 'b'), eda.And('a', 'ci'), eda.And('b', 'ci'))


sum2 = ~a & ~b & ci | ~a & b & ~ci | a & ~b & ~ci | a & b & ci
cout2 = a & b | a & ci | b & ci

write_circuit([sum1, cout1, sum2, cout2], circuit, ["sum1","cout1","sum2","cout2"])

## Equivalence check adder
circuit1 = ["sum1","cout1"]
circuit2 = ["cout2", "sum2"]
out = 2
res = equivalence_check(circuit, circuit1, circuit2, out)

dimacs :  p cnf 3 4
1 2 3 0
2 -3 -1 0
1 -3 -2 0
3 -1 -2 0
dimacs :  p cnf 3 3
1 3 0
2 3 0
1 2 0
dimacs :  p cnf 3 4
1 2 3 0
2 -3 -1 0
1 -3 -2 0
3 -1 -2 0
dimacs :  p cnf 3 3
1 3 0
2 3 0
1 2 0
bench1 :  ['sum1', 'cout1']   bench2 : ['cout2', 'sum2']
read dimacs : ((_dimacs_1 | _dimacs_2 | _dimacs_3) & (_dimacs_2 | (! _dimacs_3) | (! _dimacs_1)) & (_dimacs_1 | (! _dimacs_3) | (! _dimacs_2)) & (_dimacs_3 | (! _dimacs_1) | (! _dimacs_2)))
symbol table:  {_dimacs_1: 1, _dimacs_2: 2, _dimacs_3: 3}
read dimacs : ((_dimacs_1 | _dimacs_3) & (_dimacs_2 | _dimacs_3) & (_dimacs_1 | _dimacs_2))
symbol table:  {_dimacs_1: 1, _dimacs_2: 2, _dimacs_3: 3}
read dimacs : ((_dimacs_1 | _dimacs_3) & (_dimacs_2 | _dimacs_3) & (_dimacs_1 | _dimacs_2))
symbol table:  {_dimacs_1: 1, _dimacs_2: 2, _dimacs_3: 3}
read dimacs : ((_dimacs_1 | _dimacs_2 | _dimacs_3) & (_dimacs_2 | (! _dimacs_3) | (! _dimacs_1)) & (_dimacs_1 | (! _dimacs_3) | (! _dimacs_2)) & (_dimacs_3 | (! _dimacs_1) | (! _dimacs_2)))
symbol table:

In [29]:
a, b, c = map(eda.exprvar, 'abc')
f1 = eda.Or(eda.And(~a, ~b, ~c), eda.And(~a, ~b, c), eda.And(a, ~b, c), eda.And(a, b, c), eda.And(a, b, ~c))

In [30]:
d = circuit2dimacs(f1)
write_dimacs("test5.cnf", d)

In [5]:
a, b, c = map(exprvar, 'abc')
f1 = Or(And(~a, ~b, ~c), And(~a, ~b, c), And(a, ~b, c), And(a, b, c), And(a, b, ~c))
ex = f1.to_cnf()
exp, dimacs = expr2dimacscnf(ex)
print(dimacs)
f2 = Or(And(~a, ~b, c), And(a, ~b, c), And(a, b, c), And(a, b, ~c))
ex = f2.to_cnf()
exp, dimacs = expr2dimacscnf(ex)
print(dimacs)

In [ ]:
# vars_cnt, clauses, comments = read_dimacs('cnf_bench/test1.cnf')

# res, rev_st = dimacs.dimacs_to_pysmt(vars_cnt, clauses, comments)
# # varA = Symbol("A")
# f = Xor(res, res)

# sat_res = is_sat(f)
# # assert sat_res # SAT
# print("dimacs := %s is SAT? %s" % (res, sat_res))
# print("if UNSAT, given circuits are equal")
# # assert not sat_res # UNSAT